# Movie Sentiment Analysis
https://www.kaggle.com/c/word2vec-nlp-tutorial/

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

import pandas as pd


train = pd.read_csv('labeledTrainData.tsv', delimiter="\t")
test = pd.read_csv('testData.tsv', delimiter="\t")
train.head()                

id  sentiment                                             review
0  5814_8          1  With all this stuff going down at the moment w...
1  2381_9          1  \The Classic War of the Worlds\" by Timothy Hi...
2  7759_3          0  The film starts with a manager (Nicholas Bell)...
3  3630_4          0  It must be assumed that those who praised this...
4  9495_8          1  Superbly trashy and wondrously unpretentious 8...

In [2]:
print (train.shape)
print (test.shape)

(25000, 3)
(25000, 2)


Investigate the data

In [3]:
y_train = train['sentiment']


In [4]:
import re  

def review_to_wordlist(review):
#     print(review)

#   Keep letters
    review_text = re.sub("[^a-zA-Z]"," ", review)
#     print (review_text)
    
#   Change to lower case
    words = review_text.lower()
    
    return(words)

y_train = train['sentiment']
train_data = []

for i in range(0,len(train['review'])):
    if i % 1000 == 0:
        print ('training process line: ', str(i))
    train_data.append(review_to_wordlist(train['review'][i]))
        
train_data = np.array(train_data)
test_data = []
for i in range(0,len(test['review'])):
    if i % 1000 == 0:
        print ('testing process line: ', str(i))
    test_data.append(review_to_wordlist(test['review'][i]))
    
test_data = np.array(test_data)

training process line:  0
training process line:  1000
training process line:  2000
training process line:  3000
training process line:  4000
training process line:  5000
training process line:  6000
training process line:  7000
training process line:  8000
training process line:  9000
training process line:  10000
training process line:  11000
training process line:  12000
training process line:  13000
training process line:  14000
training process line:  15000
training process line:  16000
training process line:  17000
training process line:  18000
training process line:  19000
training process line:  20000
training process line:  21000
training process line:  22000
training process line:  23000
training process line:  24000
testing process line:  0
testing process line:  1000
testing process line:  2000
testing process line:  3000
testing process line:  4000
testing process line:  5000
testing process line:  6000
testing process line:  7000
testing process line:  8000
testing proces

In [8]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer()


data_train_count = vectorizer.fit_transform(train_data)
data_test_count  = vectorizer.transform(test_data)

tfidf = TfidfVectorizer(
            ngram_range=(1, 3),  
            use_idf=1,
            smooth_idf=1,
            stop_words = 'english') 


data_train_count_tf = tfidf.fit_transform(train_data)
data_test_count_tf  = vectorizer.transform(test_data)


In [ ]:
# print (data_train_count.shape, y_train.shape, data_test_count.shape)
# print (data_train_count_tf)


word_freq_df = pd.DataFrame({'term': tfidf.get_feature_names(), 'tfidf':data_train_count_tf.toarray().sum(axis=0)})
plt.plot(word_freq_df.occurrences)

plt.show() 


word_freq_df_sort = word_freq_df.sort_values(by=['tfidf'], ascending=False)
word_freq_df_sort.head()

In [ ]:
from sklearn.naive_bayes import MultinomialNB 

clf = MultinomialNB()
clf.fit(data_train_count, y_train)
from sklearn.model_selection import cross_val_score
import numpy as np

print ("MultinomialNB cross val score: ", np.mean(cross_val_score(clf, data_train_count, y_train, cv=10, scoring='accuracy')))

print ("MultinomialNB cross val score: ", np.mean(cross_val_score(clf, data_train_count_tf, y_train, cv=10, scoring='accuracy')))


In [ ]:
pred = clf.predict(data_test_count)
print (pred)

df = pd.DataFrame({"id": test['id'],"sentiment": pred})

df.to_csv('submission.csv',index = False, header=True)